# 1 Introduction and clinical context

# 2 Defintions methods and hypotheses

# 3 Import of data and libraries

## 3.1 Import of libraries

In [103]:
#Importing basic data analyst libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#Importing libraries for SQL querrying
import duckdb 

##3.2 Importing tables

I decided to import all the tables relevant for my analysis at this point. I will treat them as a database and later on i will look at them in detail. Now I will just import them one by one whereas querrying and cleaning comes a bit later in separate stages. I chose this approach because i find it more logically organised compared to other solutions.

###3.2.1 Importing confirmed cases table

In [108]:
df_cases = pd.read_csv('data/raw/confirmed_global.csv')

Verifying import

In [111]:
print(df_cases.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...   56517   56572   56595   56676   
1        0        0        0        0  ...  125506  125842  126183  126531   
2        0        0        0        0  ...  117304  117429  117524  117622   
3        0        0        0        0  ...   12053   12115   12174   12231   
4        0        0        0        0  ...   22399   22467   22579   22631   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0   56717   56779   56873   56943

##### ---Cases dataset imported successfully---

### 3.2.2 Importing death cases

In [115]:
df_deaths = pd.read_csv('data/raw/deaths_global.csv')

Verifying import

In [118]:
print(df_deaths.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...    2489    2495    2496    2497   
1        0        0        0        0  ...    2241    2247    2256    2265   
2        0        0        0        0  ...    3096    3099    3102    3105   
3        0        0        0        0  ...     115     116     117     117   
4        0        0        0        0  ...     538     538     540     542   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0    2508    2512    2512    2516

##### ----Deaths dataset importet successfully----

### 3.2.3 Importing recovered cases dataset

In [122]:
df_recovered = pd.read_csv('data/raw/recovered_global.csv')

In [124]:
print(df_recovered.head())

  Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  4/1/21  4/2/21  4/3/21  4/4/21  \
0        0        0        0        0  ...   51788   51798   51802   51885   
1        0        0        0        0  ...   91875   92500   93173   93842   
2        0        0        0        0  ...   81632   81729   81813   81896   
3        0        0        0        0  ...   11365   11401   11428   11474   
4        0        0        0        0  ...   20508   20867   20871   20879   

   4/5/21  4/6/21  4/7/21  4/8/21  4/9/21  4/10/21  
0   51902   51928   51940   51956

##### ----Recovered dataset importet successfully!----